In [ ]:
import pandas as pd
import numpy as np
import os, gc
os.chdir("/home/jovyan/work/phd/datasets/gtex/10")

In [ ]:
df=pd.read_csv("mainTable.csv", index_col=0)
df.info()

In [ ]:
with open("corpus.txt",'w') as file:
    for sample in df.columns:
        for g in np.array(df[sample].sort_values(ascending=False).index[:1000],dtype=str):
            file.write(g[:15])
            file.write(" ")
        file.write("\n")

In [ ]:
gc.collect()

In [ ]:
os.system("export PATH=$PATH:/home/jovyan/work/phd/topicmapping/bin")
os.system("topicmap -f corpus.txt -r 10 -t 10 -seed 42 -o tm")

In [ ]:
df_words = pd.read_csv("tm/word_wn_count.txt", sep=' ', header=None)
df_words.columns=['word', 'word-id', 'occurrence']
df_words.sort_values('word-id')

In [ ]:
df_topic_distr = pd.read_csv("tm/lda_gammas_final.txt", sep=' ', header=None)
df_topic_distr.columns=['Topic %d'%(t+1) for t in df_topic_distr.columns]
df_topic_distr.index.name='i_doc'
df_topic_distr.insert(0,'doc',df.columns)
df_topic_distr=df_topic_distr.dropna(how='all',axis=1)
df_topic_distr

In [ ]:
clusters = df_topic_distr.drop('doc',1).values.argmax(1)
df_clusters=pd.DataFrame()
for cluster in range(np.max(clusters)+1):
    elems=df.columns[clusters==cluster].values
    df_clusters.insert(0,'Cluster %d'%(cluster+1),np.concatenate([elems, ['' for _ in range(len(df.columns)- len(elems))]]))

In [ ]:
df_clusters.sort_index(axis=1).to_csv("tm/tm_level_0_clusters.csv", index=False, header=True)

In [ ]:
df_topic_distr.to_csv("tm/tm_level_0_topic-dist.csv", index=True, header=True)

In [ ]:
df_word_distr = pd.DataFrame().fillna(0)
with open("tm/lda_betas_sparse_final.txt","r") as f:
    for line in f.read().split("\n"):
        row = line.split(" ")
        if len(row) < 2:
            continue
        topic = int(row[0])+1
        line=np.array(row[1:-1], dtype=float).reshape(int((len(row)-1)/2),2)
        for el in line:
            df_word_distr.at[df_words[df_words['word-id']==int(el[0])].word.values[0], f"Topic {topic}"] = el[1]
#df_word_distr.index=df_words['word']
df_word_distr.fillna(0)
df_word_distr.to_csv("tm/tm_level_0_word-dist.csv", index=True, header=True)

In [ ]:
df_topics = pd.DataFrame()
max_L = df_word_distr.shape[0]
for topic in df_word_distr.columns[::-1]:
    t_series = df_word_distr[topic]
    t_series = t_series[t_series>t_series.quantile(0.99)]
    df_topics.insert(0,topic,np.concatenate((t_series.index.values,np.repeat(np.nan, df_word_distr.shape[0]-len(t_series)))))
df_topics.dropna(how="all", axis=0).to_csv("tm/tm_level_0_topics.csv", index=False, header=True)